In [9]:
from tensorflow import keras

model = keras.models.load_model('works_well_2')

In [10]:
VIDEO_NAME = "e3.mp4"
FRAMERATE = 30


In [11]:
import cv2

import os, shutil
image_folder = 'images/'
for filename in os.listdir(image_folder):
    file_path = os.path.join(image_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

vidcap = cv2.VideoCapture('video/' + VIDEO_NAME)
success,image = vidcap.read()
count = 0

while success:
    if count % 2 == 0:
        count+=1
        continue
    
    cv2.imwrite("images/frame%d.jpg" % count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    count += 1

In [12]:
import os

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

import numpy as np

In [13]:
IMG_SIZE = [224, 224]

Y_predict = []
X_test = []
test_filenames = []

for filename in os.listdir(image_folder):
    if not os.path.isfile(f'{image_folder}/{filename}'):
        continue
        
    imagePath = f"{image_folder}/{filename}"

    image = load_img(imagePath, target_size=IMG_SIZE)
    image = img_to_array(image)

    test_filenames.append(filename)
    X_test.append(image)

X_test = np.array(X_test, dtype="float32") / 255.0
Y_predict = np.zeros((X_test.shape[0], 4), dtype="float32")

In [14]:
Y_predict = model.predict(
    X_test,
    batch_size=None,
    verbose='auto',
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False
)

4/4 [==============================] - 8s 378ms/step


In [15]:
Y_predict

array([[0.02067842, 0.        , 0.0313585 , 0.        ],
       [0.05033438, 0.01290921, 0.0496638 , 0.        ],
       [0.05649479, 0.01979372, 0.07399588, 0.        ],
       [0.06461509, 0.02300812, 0.09068701, 0.        ],
       [0.06427105, 0.01922743, 0.09353847, 0.        ],
       [0.06719194, 0.0195465 , 0.08335629, 0.        ],
       [0.00840583, 0.        , 0.008827  , 0.        ],
       [0.06586013, 0.01548952, 0.08278102, 0.        ],
       [0.06608891, 0.01998199, 0.09375405, 0.        ],
       [0.05823354, 0.02449958, 0.08588868, 0.        ],
       [0.05439277, 0.0076702 , 0.08071786, 0.        ],
       [0.06000926, 0.00431407, 0.07500877, 0.        ],
       [0.06623101, 0.00261914, 0.07264856, 0.        ],
       [0.0539212 , 0.        , 0.05659514, 0.        ],
       [0.05116263, 0.        , 0.04889837, 0.        ],
       [0.04838473, 0.00124971, 0.03822332, 0.        ],
       [0.04573258, 0.00467196, 0.03769713, 0.        ],
       [0.00698095, 0.        ,

In [16]:
from PIL import Image, ImageDraw

img_array = []
size = []

import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

for i, filename in enumerate(sorted_alphanumeric(os.listdir(image_folder))):
    img = Image.open(f"{image_folder}/{filename}")
    draw = ImageDraw.Draw(img)
    
    w, h = img.size
    
    xmin = Y_predict[i][0] * w
    xmax = Y_predict[i][2] * w
    ymin = Y_predict[i][1] * h
    ymax = Y_predict[i][3] * h
    
    draw.line((xmin, ymin, xmax, ymin), fill=(0, 255, 0))
    draw.line((xmax, ymin, xmax, ymax), fill=(0, 255, 0))
    draw.line((xmin, ymin, xmin, ymax), fill=(0, 255, 0))
    draw.line((xmax, ymax, xmin, ymax), fill=(0, 255, 0))
    
    size = (w, h)
    img_array.append(np.asarray(img))

out = cv2.VideoWriter('result.mp4', 0, FRAMERATE, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()